In [1]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
import os
from dotenv import load_dotenv
import pandas as pd

# Load environment variables
load_dotenv()

True

In [2]:
# Use environment variables
SERVICE_FILE = os.getenv('SERVICE_FILE_PATH')
SPREADSHEET_ID = os.getenv('SPREADSHEET_ID')
RANGE_NAME = os.getenv('RANGE_NAME')

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

In [3]:
def get_credentials():
    """Authenticate using the service account."""
    return service_account.Credentials.from_service_account_file(SERVICE_FILE, scopes=SCOPES)


In [4]:
def read_sheet_data(service, spreadsheet_id, range_name):
    """Reads data from a specified range in the sheet and returns it as a pandas DataFrame."""
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])

    # Check if there's any data fetched from the sheet
    if not values:
        print('No data found.')
        return pd.DataFrame()  # Return an empty DataFrame if no data is found
    else:
        df = pd.DataFrame(values[1:], columns=values[0])  # Create a DataFrame, assuming the first row is the header
        return df

def write_data_to_sheet(service, spreadsheet_id, range_name, data_frame):
    """Writes data from a pandas DataFrame to the sheet."""
    values = [data_frame.columns.tolist()] + data_frame.values.tolist()  # Include headers
    body = {'values': values}
    service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name,
                                           valueInputOption='USER_ENTERED', body=body).execute()
    print(f"Data written to sheet starting at {range_name}.")

In [5]:
service = build('sheets', 'v4', credentials=get_credentials())

In [6]:
# Read data
df = read_sheet_data(service, SPREADSHEET_ID, RANGE_NAME)

# Do something with the DataFrame
if not df.empty:
    print(df)
else:
    print("The DataFrame is empty.")

             Sub-team headcount
0   Cavs - Propulsion         5
1       Cavs - System         3
2      Cavs - another        12
3       Comm - social         3
4     Comm - outreach         0
5      Comm - another         4
6     PM - automation         6
7         EiM - team1         2
8         Eim - team2         5
9         SDI - team1         8
10        SDI - team2         0
11        SDI - team3         2
12        PCM - team1         5
13        PCM - team2        10
14        PCM - team2         2


In [7]:
# Clear the sheet or range
# clear_sheet(service, SPREADSHEET_ID, RANGE_NAME)

In [8]:
# Load data into DataFrame and write to sheet
df = pd.read_csv('headcount.csv')
write_data_to_sheet(service, SPREADSHEET_ID, RANGE_NAME + '!A1', df)

Data written to sheet starting at Sheet1!A1.
